I'm currently plagued with submission errors.

This error must be caused by my "create_daily_lag_feature" function　to create target shift feature group by player.
I am sure that the error is caused by this function. But I have no idea why.

When I check the sample iteration test, It doesn't cause any errors. But only when I submit it, caused Notebook Threw Exception.


I tried a lot of checking(reduce_memory, change dtypes, reduce dataframe rows), but none of it has helped

Please help me.

In [ ]:
# ----- Import common library -----　
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
from tqdm import tqdm_notebook as tqdm
from glob import glob
import gc
import pickle
from time import time, sleep
import json
import pytz
import random
pd.set_option('display.max_columns', 500)
import warnings
warnings.filterwarnings('ignore')
from IPython.core.display import display
import tempfile

In [ ]:
import numpy as np
from tqdm import tqdm
from pandas.api.types import is_datetime64_any_dtype as is_datetime
from pandas.api.types import is_categorical_dtype
from tqdm import tqdm_notebook as tqdm

def reduce_mem_usage(df, use_float16=False):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in tqdm(df.columns):
        if is_datetime(df[col]) or is_categorical_dtype(df[col]) or str(df[col].dtype)=='timedelta64[ns]':
            # skip datetime type or categorical type
            continue
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if use_float16 and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            #df[col] = df[col].astype('category')
            df[col] = df[col].astype(str)

    end_mem = df.memory_usage().sum() / 1024**2
    print ('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print ('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [ ]:
def read_data():
    train = pd.read_pickle('../input/mlb-engagement-preprocessed/train_nextDayPlayerEngagement.pkl')
    train['date'] = pd.to_datetime(train['engagementMetricsDate']).map(lambda x: x - timedelta(days=1))
    train.drop(columns=['engagementMetricsDate'], inplace=True)
    
    #playerTwitterFollowers = pd.read_pickle('../input/local_eval/train_playerTwitterFollowers.pkl')
    
    teams = pd.read_csv('../input/mlb-player-digital-engagement-forecasting/teams.csv')
    awards = pd.read_csv('../input/mlb-player-digital-engagement-forecasting/awards.csv')
    seasons = pd.read_csv('../input/mlb-player-digital-engagement-forecasting/seasons.csv')
    players = pd.read_csv('../input/mlb-player-digital-engagement-forecasting/players.csv')
    example_test = pd.read_csv('../input/mlb-player-digital-engagement-forecasting/example_test.csv') 
    example_sample_submission = pd.read_csv('../input/mlb-player-digital-engagement-forecasting/example_sample_submission.csv')

    return train, teams, awards, seasons, players, example_test, example_sample_submission

df, teams, awards, seasons, players, example_test, example_sample_submission = read_data()
df

In [ ]:
def create_daily_lag_feature(df, datetime_col, groupby_cols, target_col, lags=[1]):
    df_copy = df[[datetime_col, target_col] + groupby_cols].copy()
    new_cols = []
    for lag in tqdm(lags):
        new_col = target_col + '_' + '_'.join(groupby_cols) + f'_lag{lag}days'
        if new_col in df.columns:
            df.drop(columns=new_col, inplace=True)
        lag_df = df_copy.set_index(datetime_col).groupby(groupby_cols)[target_col].shift(
            freq=f'{lag}D').reset_index().rename(columns={target_col: new_col})
        df = df.merge(lag_df, on=[datetime_col] + groupby_cols, how='left')
        new_cols.append(new_col)
    del df_copy
    gc.collect()
    return df, new_cols

In [ ]:
import time
start_time = time.time()

target_cols = ['target1', 'target2', 'target3', 'target4']
for target in target_cols:
    df, new_cols = create_daily_lag_feature(
        df=df,
        datetime_col='date', 
        groupby_cols=['playerId'], 
        target_col=target, 
        lags=[31, 32, 33, 34, 35, 36, 37,]
    )

df = reduce_mem_usage(df)

train_df = df.reset_index(drop=True).copy()

del df
gc.collect()

end_time = time.time()
print ('{} sec'.format(end_time-start_time))

train_df

In [ ]:
import mlb
env = mlb.make_env() # initialize the environment
iter_test = env.iter_test() # iterator which loops over each date in test set

target_cols = ['target1', 'target2', 'target3', 'target4']
required_days = 37

for (test_df, sample_prediction_df) in iter_test:
    
    #display(test_df)
    #display(sample_prediction_df)
    
    test_start_date = test_df.index.map(lambda x: datetime.strptime(str(x), '%Y%m%d')).unique()[0]
    train_df = train_df[train_df['date']<test_start_date]
    train_start_date = test_start_date - timedelta(days=required_days)
    train_df = train_df[train_df['date']>=train_start_date]
    #train_df = train_df.sort_values(['date', 'playerId']).reset_index(drop=True)
    
    sub_df = sample_prediction_df.copy()
    sample_prediction_df = sample_prediction_df.reset_index()
    sample_prediction_df['date'] = pd.to_datetime(sample_prediction_df['date'].map(lambda x: datetime.strptime(str(x), '%Y%m%d')))
    sample_prediction_df['playerId'] = sample_prediction_df['date_playerId'].map(lambda x: int(x.split('_')[1]))
    
    train_df['test'] = 0
    sample_prediction_df['test'] = 1
    df = pd.concat([train_df, sample_prediction_df])
    df = df.drop_duplicates(['date', 'playerId']).reset_index(drop=True)
    df = reduce_mem_usage(df)
    del train_df, sample_prediction_df
    gc.collect()

    df['playerId'] = df['playerId'].astype(str)
    df[target_cols] = df[target_cols].astype(np.float64)
    
    """
    This following block caused Submission Error(Notebook Threw Exception) only when submission. No error will occur when sample-prediction.
    If this block is commented out, it does not caused Submission Error, the score will be calculated normally (all zero submission score)
    
    Even if lags = [1], it happens same error. So I wonder May dataset has some difference with public dataset.
    
    WHY???
    """
    for target in target_cols:
        df, new_cols = create_daily_lag_feature(
            df=df,
            datetime_col='date', 
            groupby_cols=['playerId'], 
            target_col=target, 
            lags=[31, 32, 33, 34, 35, 36, 37,]
        )
    """"""
        
        
    test = df[df['test']==1].reset_index(drop=True)
    #display(test_df)
    
    test_preds = np.zeros((len(test), 4)).astype(np.float64)
    
    for seed in tqdm([2021]):
        for fold in range(1):
            for i, target in enumerate(target_cols):
                test_x = test.copy()
                preds = np.zeros((len(test_x), )).astype(np.float64)
                test_preds[:, i] += preds / (len([2021]) * 1)
    
    train_df = df.copy()
    del df
    gc.collect()
    
    sub_df[target_cols] = np.clip(test_preds, 0, 100).astype(np.float64)
    sub_df[target_cols] = sub_df[target_cols].fillna(0.)
    env.predict(sub_df)
    gc.collect()
    #break

In [ ]:
sub_df